In [96]:
from tqdm import tqdm
from collections import defaultdict
import os

import fasttext
import nltk

file_dir = '../data/processed'

file_list = [
    'all_book.txt',
    'apkbr_ru.txt',
    'elgkbr_ru.txt',
    'oshhamaho.txt',
]


def clen_sentence(sentence):
    return ' '.join([word for word in nltk.word_tokenize(sentence) if len(word) > 2])

model = fasttext.load_model('../data/processed/lang_detect/models/rus_0.86__kbd_0.98__ady_0.97.model')

sentences_by_lang = defaultdict(set)
for file in file_list:
    with open(os.path.join(file_dir, file), 'r') as f:
        sentences = f.read().split('\n')

    for sent in tqdm(sentences):
        if len(sent) > 10:
            sent = clen_sentence(sent)
            lang, score = model.predict(sent)
            lang, score = lang[0], score[0]
            if score > 0.9:
                sentences_by_lang[lang].add(sent)
            else:
                sentences_by_lang['__label__mixed'].add(sent)

100%|██████████| 163472/163472 [00:18<00:00, 8961.54it/s] 


In [97]:
[(lang, len(sentences)) for lang, sentences in sentences_by_lang.items()]

[('__label__mixed', 93961),
 ('__label__kbd', 931136),
 ('__label__rus', 431710),
 ('__label__ady', 135421)]

In [98]:
sentences_by_lang['__label__rus']

{'Недавно IотдельнымI домом заживший гостям',
 'УКкАЗАТЕЛЬ Кытайсгий',
 'Радость коне примчалась',
 'попимаются всо гдаголы имеющие значение общенин помощью речи',
 'ЛлЛЫЦЫКУЛЫБЖЬ Блюдо мяса Варят мясо затем руками разрывают его мелкие кусочки масле обжаривают болышое',
 'университетской среде вне Ленинграда проторил было дорогу новому учению',
 'Отрицательные формы финитных глаголов адыгейского',
 'тивный перезэ предложение',
 'Флота Его главная задача привлечь писателей разработке оборонной темы Горький предупреждал живем враждебном окружении под',
 'къIз кэIа къIэпIэ къIапIэ рука',
 'свежей мысли з1с никогда высказал тех для нас несуразных',
 'существенные различия позволяют-выделить два основных варианта свадъбы Один них-был представлен только свадебной обрядностью ст. Прохладной второй половины ХIХ Основные',
 'этой выписки текстом статай Постановления',
 'прославиться силой ловкостью было одним первостепенных желаний как этом свидетельствовал первой половине ХIХ века Лонгворт пис

In [99]:
output_dir = '../data/processed/sentences_by_lang'
os.makedirs(output_dir, exist_ok=True)

for lang, sentences in sentences_by_lang.items():
    lang = lang.replace('__label__', '')
    
    with open(os.path.join(output_dir, f'{lang}.txt'), 'w') as f:
        f.write('\n'.join(sorted(sentences)))

In [100]:
ady_alpha = {
    'шъ',
    'шъ',
    'шIо',
    'гъо',
    'жъ',
}

kbd_alpha = {
    'I',
    'къ',
    'гъ',
    'уэ',
    'уы',
    'хъ',
    'хь',
    'зэ',
    'лъ',
    'хы',
    'хэ',
}

exact_ady_sents = set()
for ady_sent in sentences_by_lang['__label__ady']:
    for alpha in ady_alpha:
        exact_ady_sents.add(ady_sent)

exact_rus_sents = set()
for rus_sent in sentences_by_lang['__label__rus']:
    if any([alpha in rus_sent for alpha in kbd_alpha | ady_alpha]):
        continue
    exact_rus_sents.add(rus_sent)

exact_kbd_sents = set()
for kbd_sent in sentences_by_lang['__label__kbd']:
    if any([alpha in kbd_sent for alpha in ady_alpha]):
        continue
    
    exact_kbd_sents.add(kbd_sent)

In [101]:
with open('../data/processed/lang_detect/collect.sentences.ady.txt', 'w') as f:
    f.write('\n'.join(sorted(exact_ady_sents)))

In [102]:
with open('../data/processed/lang_detect/collect.sentences.rus.txt', 'w') as f:
    f.write('\n'.join(sorted(exact_rus_sents)))

In [103]:
with open('../data/processed/lang_detect/collect.sentences.kbd.txt', 'w') as f:
    f.write('\n'.join(sorted(exact_kbd_sents)))

In [104]:
from collections import Counter

rus_words_cnt = Counter(nltk.word_tokenize('\n'.join(exact_rus_sents)))
ady_words_cnt = Counter(nltk.word_tokenize('\n'.join(exact_ady_sents)))
kbd_words_cnt = Counter(nltk.word_tokenize('\n'.join(exact_kbd_sents)))

In [105]:
rus_words = set([word for word, freq in rus_words_cnt.most_common(50000)])
ady_words = set([word for word, freq in ady_words_cnt.most_common(50000)])
kbd_words = set([word for word, freq in kbd_words_cnt.most_common(50000)])

In [106]:
with open('../data/processed/lang_detect/collect.words.ady.txt', 'w') as f:
    ady_valid_words = sorted(ady_words - kbd_words - rus_words)
    ady_valid_words = ady_valid_words[ady_valid_words.index('Iабгъу'):]  # slice by first valid word
    f.write('\n'.join(ady_valid_words))

with open('../data/processed/lang_detect/collect.words.rus.txt', 'w') as f:
    rus_valid_words = sorted(rus_words - kbd_words - ady_words)
    rus_valid_words = rus_valid_words[rus_valid_words.index('Абадзех'):]  # slice by first valid word
    f.write('\n'.join(rus_valid_words))

with open('../data/processed/lang_detect/collect.words.kbd.txt', 'w') as f:
    kbd_valid_words = sorted(kbd_words - rus_words - ady_words)
    kbd_valid_words = kbd_valid_words[kbd_valid_words.index('Iеижь'):]  # slice by first valid word
    f.write('\n'.join(kbd_valid_words))